In [1]:
%load_ext autotime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import time
import simfin as sf

from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

time: 1.41 s (started: 2022-10-08 10:00:25 +08:00)


In [2]:
plt.rcParams['figure.figsize'] = [7.0, 4.5]
plt.rcParams['figure.dpi'] = 150

time: 0 ns (started: 2022-10-08 10:00:27 +08:00)


In [3]:
# Read in stock price fundamental ratios
X = pickle.load(open('./data/fundamentals3.pkl', 'rb'))

# Read in annual performance
y = pickle.load(open('./data/performance3.pkl', 'rb'))
y = y['Returns']

# Read in annual performance
y_withdata = pickle.load(open('./data/performance2.pkl', 'rb'))

time: 31 ms (started: 2022-10-08 10:00:27 +08:00)


In [4]:
X.head()

,EV_EBIT,OI_NWCFA,PE,PB,PS,OI_IntExp,WorkingCap,ROE,ROCE,DE,DebtRatio,CashRatio,AssetTurnover,GPM,WC_TA,RE_TA,EBIT_TA,Equity_Liab
0,82.797844,0.054971,112.795556,3.347207,12.792295,11.437500,3.846561,0.029675,0.024383,0.835610,2.196731,2.422222,1.738654,0.529253,0.345137,0.781242,0.021427,1.196731
1,73.741818,0.064395,93.220476,3.642102,14.677638,12.875000,3.337925,0.039070,0.030812,0.830698,2.203807,2.057851,1.633997,0.537957,0.323425,-0.057546,0.026550,1.203807
2,85.881068,0.061131,109.984756,4.122857,16.367967,13.400000,3.201348,0.037486,0.030165,0.832229,2.201593,2.012637,1.632593,0.537205,0.325973,-0.049027,0.025699,1.201593
3,93.118058,0.057908,112.367543,4.264654,17.651993,15.461538,3.219984,0.037953,0.029345,0.791585,2.263288,2.065270,1.555866,0.535009,0.333495,-0.031473,0.024936,1.263288
4,89.891130,0.063609,125.146780,4.581382,18.629924,17.923077,3.300871,0.036608,0.033366,0.742709,2.346422,2.120348,1.570674,0.544155,0.344885,-0.014954,0.028365,1.346422


time: 16 ms (started: 2022-10-08 10:00:27 +08:00)


In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38307 entries, 0 to 38306
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EV_EBIT        38307 non-null  float64
 1   OI_NWCFA       38307 non-null  float64
 2   PE             38307 non-null  float64
 3   PB             38307 non-null  float64
 4   PS             38307 non-null  float64
 5   OI_IntExp      38307 non-null  float64
 6   WorkingCap     38307 non-null  float64
 7   ROE            38307 non-null  float64
 8   ROCE           38307 non-null  float64
 9   DE             38307 non-null  float64
 10  DebtRatio      38307 non-null  float64
 11  CashRatio      38307 non-null  float64
 12  AssetTurnover  38307 non-null  float64
 13  GPM            38307 non-null  float64
 14  WC_TA          38307 non-null  float64
 15  RE_TA          38307 non-null  float64
 16  EBIT_TA        38307 non-null  float64
 17  Equity_Liab    38307 non-null  float64
dtypes: flo

In [6]:
y.head()

0    0.563351
1    0.535595
2    0.198018
3    0.076426
4   -0.050044
Name: Returns, dtype: object

time: 0 ns (started: 2022-10-08 10:00:27 +08:00)


In [7]:
y.isnull().sum()

8657

time: 0 ns (started: 2022-10-08 10:00:27 +08:00)


In [8]:
y_withdata.head()

,Ticker,Date1,Open1,Volume1,Date2,Open2,Volume2
0,A,2016-10-31,43.33,1556762.0,2017-10-31,67.74,1609938.0
1,A,2017-01-31,48.04,2552612.0,2018-01-31,73.77,1949108.0
2,A,2017-04-30,55.50,1897018.0,2018-04-30,66.49,2277705.0
3,A,2017-07-31,60.32,1038744.0,2018-07-31,64.93,2539265.0
4,A,2017-10-31,67.74,1609938.0,2018-10-31,64.35,3820987.0


time: 0 ns (started: 2022-10-08 10:00:27 +08:00)


In [9]:
# X AND Y (splitting for train/test done previously for trained model)
X_train=pd.read_csv("Annual_Stock_Price_Fundamentals_Ratios_train.csv", 
                    index_col=0)
X_test=pd.read_csv("Annual_Stock_Price_Fundamentals_Ratios_test.csv", 
                   index_col=0)
y_train=pd.read_csv("Annual_Stock_Price_Performance_Percentage_train.csv", 
                    index_col=0)['Returns']
y_test=pd.read_csv("Annual_Stock_Price_Performance_Percentage_test.csv", 
                   index_col=0)['Returns']

time: 109 ms (started: 2022-10-08 10:00:27 +08:00)


In [10]:
rf = pickle.load(open('rf_regressor.p', 'rb'))

time: 15 ms (started: 2022-10-08 10:00:27 +08:00)


In [11]:
# fit and predict using Random Forest
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

time: 17 s (started: 2022-10-08 10:00:27 +08:00)


In [12]:
mse_train = mean_squared_error(y_train, rf.predict(X_train), squared=True)
mse_test = mean_squared_error(y_test, y_pred, squared=True)
pct_diff = round((1-mse_test/mse_train)*100,3)

print('Train MSE:', mse_train)
print('Test MSE:', mse_test)
print(f'Generalisation: {pct_diff}%')

Train MSE: 0.6734017695836776
Test MSE: 1.017684904552254
Generalisation: -51.126%
time: 156 ms (started: 2022-10-08 10:00:44 +08:00)


In [13]:
y_withdata_test = pd.DataFrame()
y_withdata_test = y_withdata.loc[y_test.index, :]

time: 0 ns (started: 2022-10-08 10:00:44 +08:00)


In [14]:
y_withdata_test

,Ticker,Date1,Open1,Volume1,Date2,Open2,Volume2
26780,PK,2016-12-31,30.21,33221775.0,2017-12-31,28.87,1294841.0
16361,HTGM,2021-06-30,5.99,27359.0,2022-06-30,NaN,NaN
27115,PNW,2020-12-31,79.05,539364.0,2021-12-31,NaN,NaN
18629,KE,2018-06-30,18.25,53290.0,2019-06-30,16.25,67064.0
258,ABIO,2021-06-30,3.52,92318.0,2022-06-30,NaN,NaN
...,...,...,...,...,...,...,...
22,AA,2017-06-30,33.16,3014777.0,2018-06-30,46.46,2150764.0
25249,OLLI,2019-05-31,98.38,727763.0,2020-05-30,91.84,2767397.0
5903,CALA,2020-09-30,71.50,23076.0,2021-09-30,43.20,38780.0
19675,LFVN,2017-12-31,4.79,8906.0,2018-12-31,13.04,119735.0


time: 15 ms (started: 2022-10-08 10:00:44 +08:00)


In [15]:
# Set your SimFin+ API-key for downloading data.
sf.set_api_key('nYdjeL237mt7QaQ4OZTaPa4Xn3YECrTH')

# Set the local directory where data-files are stored.
# The directory will be created if it does not already exist.
sf.set_data_dir('~/simfin_data/')

time: 0 ns (started: 2022-10-08 10:00:44 +08:00)


In [16]:
# Download the data from the SimFin server and load into a Pandas DataFrame.
df_prices = sf.load_shareprices(variant='daily', market='us')
df_prices.reset_index(inplace=True)

Dataset "us-shareprices-daily" on disk (0 days old).
- Loading from disk ... Done!
time: 10 s (started: 2022-10-08 10:00:44 +08:00)


In [17]:
# Print the first rows of the data.
df_prices.head()

,Ticker,Date,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
0,A,2016-10-06,45846,47.01,46.88,47.43,47.30,45.09,NaN,1772801,324384755.0
1,A,2016-10-07,45846,47.33,46.99,47.49,47.38,45.16,NaN,1663127,324384755.0
2,A,2016-10-10,45846,47.58,47.55,48.15,47.69,45.46,NaN,1544958,324384755.0
3,A,2016-10-11,45846,47.19,45.42,47.19,45.69,43.55,NaN,2912537,324384755.0
4,A,2016-10-12,45846,45.78,45.42,45.81,45.47,43.34,NaN,1736076,324384755.0


time: 16 ms (started: 2022-10-08 10:00:54 +08:00)


In [18]:
df_prices.shape

(3172869, 11)

time: 31 ms (started: 2022-10-08 10:00:54 +08:00)


# Do Functions here

In [19]:
def getYPerf(y_):
    y=pd.DataFrame()
    y["Ticker"] = y_["Ticker"]
    y["Perf"]=(y_["Open2"]-y_["Open1"])/y_["Open1"]
    y[y["Perf"].isnull()]=0
    return y

time: 0 ns (started: 2022-10-08 10:00:54 +08:00)


In [20]:
y_withdata_test

,Ticker,Date1,Open1,Volume1,Date2,Open2,Volume2
26780,PK,2016-12-31,30.21,33221775.0,2017-12-31,28.87,1294841.0
16361,HTGM,2021-06-30,5.99,27359.0,2022-06-30,NaN,NaN
27115,PNW,2020-12-31,79.05,539364.0,2021-12-31,NaN,NaN
18629,KE,2018-06-30,18.25,53290.0,2019-06-30,16.25,67064.0
258,ABIO,2021-06-30,3.52,92318.0,2022-06-30,NaN,NaN
...,...,...,...,...,...,...,...
22,AA,2017-06-30,33.16,3014777.0,2018-06-30,46.46,2150764.0
25249,OLLI,2019-05-31,98.38,727763.0,2020-05-30,91.84,2767397.0
5903,CALA,2020-09-30,71.50,23076.0,2021-09-30,43.20,38780.0
19675,LFVN,2017-12-31,4.79,8906.0,2018-12-31,13.04,119735.0


time: 15 ms (started: 2022-10-08 10:00:54 +08:00)


time: 0 ns (started: 2022-10-08 10:00:54 +08:00)


time: 0 ns (started: 2022-10-08 10:00:54 +08:00)


## Functions tested and working

### Function 1: Get stock price between dates

In [24]:
# To get the stock price between 2 dates for a particular ticker
def getStockPriceBetweenDates(date1, date2, ticker, df_prices):
    rows = df_prices.loc[(df_prices["Date"]>date1) & 
                         (df_prices["Date"]<date2) & 
                         (df_prices["Ticker"]==ticker)]
    return rows

time: 0 ns (started: 2022-10-08 10:00:54 +08:00)


In [25]:
# Testing above function
getStockPriceBetweenDates('2016-10-09', '2016-10-15', 'AAPL', df_prices)

,Ticker,Date,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
10283,AAPL,2016-10-10,111052,28.75,28.68,29.19,29.01,27.05,NaN,144943824,2.134466e+10
10284,AAPL,2016-10-11,111052,29.43,29.05,29.67,29.07,27.11,NaN,256164172,2.134466e+10
10285,AAPL,2016-10-12,111052,29.34,29.19,29.50,29.34,27.35,NaN,150347148,2.134466e+10
10286,AAPL,2016-10-13,111052,29.20,28.93,29.36,29.25,27.27,NaN,140769624,2.134466e+10
10287,AAPL,2016-10-14,111052,29.47,29.28,29.54,29.41,27.42,NaN,142608764,2.132925e+10


time: 266 ms (started: 2022-10-08 10:00:54 +08:00)


### Function 2: Get stock price data

In [26]:
# Expansion of the above
# Will do a filter on daily stock prices to get stock price we want on the days we want
def getStockPriceData(ticker, y_, mask, daily_stock_prices):
    date1 = y_[mask][y_[mask]["Ticker"] == ticker]["Date1"].values[0]
    date2 = y_[mask][y_[mask]["Ticker"] == ticker]["Date2"].values[0]
    rows = getStockPriceBetweenDates(date1, date2, ticker, daily_stock_prices)
    return rows

time: 0 ns (started: 2022-10-08 10:00:55 +08:00)


In [27]:
datemask = y_withdata_test["Date1"].between(\
            pd.to_datetime('2016-11-11') - pd.Timedelta(days=60),\
            pd.to_datetime('2016-11-11') + pd.Timedelta(days=60) )

time: 0 ns (started: 2022-10-08 10:00:55 +08:00)


In [31]:
a_prices = getStockPriceData('A', y_withdata_test, datemask, df_prices)
a_prices

,Ticker,Date,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
18,A,2016-11-01,45846,43.73,43.14,44.11,43.49,41.46,NaN,2133116,324384755.0
19,A,2016-11-02,45846,43.47,43.24,43.73,43.26,41.24,NaN,2240428,324384755.0
20,A,2016-11-03,45846,43.33,43.18,44.00,43.66,41.62,NaN,2510054,324384755.0
21,A,2016-11-04,45846,43.69,43.69,44.91,44.32,42.25,NaN,2220908,324384755.0
22,A,2016-11-07,45846,44.99,44.95,45.76,45.71,43.57,NaN,1723957,324384755.0
...,...,...,...,...,...,...,...,...,...,...,...
264,A,2017-10-24,45846,67.31,66.88,67.55,67.32,64.79,NaN,1966591,321828003.0
265,A,2017-10-25,45846,67.90,67.03,68.52,67.39,64.86,NaN,1590110,321828003.0
266,A,2017-10-26,45846,67.53,67.20,67.83,67.71,65.17,NaN,1417947,321828003.0
267,A,2017-10-27,45846,67.81,67.41,68.01,67.97,65.42,NaN,1440956,321828003.0


time: 156 ms (started: 2022-10-08 10:03:19 +08:00)


In [ ]:
y_withdata_test[datemask][y_withdata_test[datemask]['Ticker']=='A']

### Function 3: Get data for date range

In [80]:
# Weekly date index is the weekly dates and rows are the stock price data

def getDataForDateRange(weekly_date_index, rows):
    ''' 
    Given a date range(index), and a series of rows, that may not correspond exactly,
    return a DataFrame that gets rows data, for each period in the date range(index)
    ''' 
    # Create an empty dataframe to store the weekly stock data later
    WeeklyStockDataRows = pd.DataFrame()

    for i in weekly_date_index:
        WeeklyStockDataRows.append(rows.iloc[rows.index.get_loc(i, method="nearest")])
    return WeeklyStockDataRows

time: 0 ns (started: 2022-10-08 11:38:02 +08:00)


### Function 4: Get Stock Time Series

In [102]:
# Returns the rows from daily_stock_prices of the stocks and years that we want

def getStockTimeSeries(dateTimeIndex, y_withData, tickers, mask, daily_stock_prices):
    ''' 
    Get the stock price as a time series DataFrame for a list of tickers.
    A mask is used to only consider stocks for a certain period.
    dateTimeIndex is typically a weekly index, so we know what days to fetch the price for.
    '''
    stockReturns = pd.DataFrame(index=dateTimeIndex)
    dTI_new = dateTimeIndex.strftime('%Y-%m-%d') # Change Date Format
    rows = pd.DataFrame()
    
    for tick in tickers:
        # Here "rows" is stock price time series data for individual stock
        rows = getStockPriceData(tick, y_withData, mask, daily_stock_prices)
        
        rows.index = pd.DatetimeIndex(rows["Date"])

        # Gets our stock price data to be in a weekly format, using the closest day if the data for a day we want is missing.
        WeeklyStockDataRows = getDataForDateRange(dTI_new, rows)

        # Here can use Open, Close, Adj. Close, etc. price
        stockReturns[tick] = WeeklyStockDataRows["Close"].values
        
    return stockRet

time: 0 ns (started: 2022-10-08 13:30:04 +08:00)


### Function 5: Get portfolio relative time series (Start from base level of 100)

In [107]:
def getPortfolioRelativeTimeSeries(stockRet):
    ''' Takes DataFrame of stock returns, one column per stock
    Normalises all the numbers so the price at the start is 1.
    Adds a column for the portfolio value.
    '''
    for key in stockRet.keys():
        stockRet[key] = stockRet[key]/stockRet[key][0]
            
    stockRet["Portfolio"] = stockRet.sum(axis=1)/(stockRet.keys().shape[0])
    return stockRet

time: 0 ns (started: 2022-10-08 13:56:18 +08:00)


### Function 6: Get Portfolio Time Series for the year

In [91]:
def getPortTimeSeriesForYear(date_starting, y_withData, X, daily_stock_prices, ml_model_pipeline):
    '''
    Function runs a backtest.
    Returns DataFrames of selected stocks/portfolio performance, for 1 year.
    y_withData is annual stock performances (all backtest years)
    date_starting e.g. '2010-01-01'
    daily_stock_prices is daily(mostly) stock price time series for all stocks
    '''
    # Get y dataframe with ticker and ticker performance only
    y = getYPerf(y_withData)
    
    # Get performance only for time frame we care about, mask original data using the start date
    thisYearMask = y_withData["Date1"].between(\
            pd.to_datetime(date_starting) - pd.Timedelta(days=60),\
            pd.to_datetime(date_starting) + pd.Timedelta(days=60) )
    
    # Get return prediction from model
    y_pred = ml_model_pipeline.predict(X[thisYearMask])
    
    # Make it a DataFrame to select the top picks
    y_pred = pd.DataFrame(y_pred)
    
    # Bool list of top stocks
    bl_bestStocks = (y_pred[0]>y_pred.nlargest(8,0).tail(1)[0].values[0]) 
    
    # DatetimeIndex (weekly)
    dateTimeIndex = pd.date_range(start=date_starting, periods=52, freq='W')
    
    # 7 greatest performance stocks of y_pred 
    ticker_list = y[thisYearMask].reset_index(drop=True)\
                  [bl_bestStocks]["Ticker"].values
    
    # Issue with one of the tickers equaling 0, fix with lambda function
    #ticker_list = list(filter(lambda dateTimeIndex: dateTimeIndex != 0, ticker_list))

    # After we know our stock picks, we get the stock performance
    # Get DataFrame index of time stamp, series of stock prices, keys=tickers
    stockRet = getStockTimeSeries(dateTimeIndex, y_withData, 
                                  ticker_list, thisYearMask, 
                                  daily_stock_prices)
    
    # Get DataFrame of relative stock prices from 
    # 1st day(or close) and whole portfolio
    stockRetRel = getPortfolioRelativeTimeSeries(stockRet)
    return [stockRetRel, stockRetRel["Portfolio"], ticker_list]

time: 0 ns (started: 2022-10-08 11:51:39 +08:00)


### Function 7: Get portfolio time series

In [108]:
def getPortTimeSeries(y_withData, X, daily_stock_prices, ml_model_pipeline, verbose=True):

    # Set date range to make stock picks over
    date_range = pd.date_range(start='2016-01-01', periods=5, freq='Y')
    
    # For each date in the date_range, make stock selections
    # and plot the return results of those stock selections
    port_perf_all_years = pd.DataFrame()
    
    perfRef=1 # Performance starts at 1.
    
    for curr_date in date_range:
        [comp, this_year_perf, ticker_list] = getPortTimeSeriesForYear(curr_date, y_withData, X,
                                                                       daily_stock_prices, ml_model_pipeline)
        
        if verbose: # If you want text output
            print("Backtest performance for year starting ", curr_date, " is:",\
                  round((this_year_perf.iloc[-1]-1)*100,2), "%")
            print("With stocks:", ticker_list)
        
        for tick in ticker_list:
            print(tick, "Performance was:", round((comp[tick].iloc[-1]-1)*100,2), "%" )
        print("---------------------------------------------")
        
        # Stitch performance for every year together
        this_year_perf = this_year_perf * perfRef
        port_perf_all_years = pd.concat([port_perf_all_years, this_year_perf])
        perfRef = this_year_perf.iloc[-1]

    # Return portfolio performance for all years
    port_perf_all_years.columns = ["Indexed Performance"]
        
    return port_perf_all_years

time: 0 ns (started: 2022-10-08 13:57:31 +08:00)


In [109]:
getPortTimeSeries(y_withdata_test, X_test, df_prices, rf)

KeyError: 'Close'

time: 391 ms (started: 2022-10-08 13:57:33 +08:00)


In [94]:
rf

RandomForestRegressor(max_depth=10, random_state=42)

time: 16 ms (started: 2022-10-08 11:53:18 +08:00)


## Above here functions are working

In [ ]:
y_withdata_test

In [ ]:
y_withdata_test[thisYearMask][y_withdata_test[thisYearMask]["Ticker"]=='CRI']["Date2"].values[0]

In [ ]:
getStockPriceData('A', y_withdata_test, thisYearMask, df_prices)

In [ ]:
dateTimeIndex = pd.date_range(start='2016-10-09', periods=52, freq='W')
getStockTimeSeries(dateTimeIndex, y_withdata_test, ['AAPL'], True, df_prices)